In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import hashlib
import time
from sqlalchemy import text, inspect, MetaData, Table, func
from sqlalchemy.orm import sessionmaker
#from dbwrapper.main import getTables, Base, Account, Transaction, TransactionFinal, Load
from dbwrapper import engine
from dbwrapper import Base, Account, Transaction, TransactionFinal, Load
from dbwrapper.main import getTables
from dbwrapper.helpers.readCsv import openOneCsv, getAccountHistory, setDateColumns
from dbwrapper.helpers.sqlEngine import engine
from dbwrapper.helpers.summarize import summarize
from dbwrapper.helpers.query_to_df import query_to_df

from dbwrapper.crud.delete.truncate_all import truncate_all
from dbwrapper.crud.read.select_accounts_df import select_accounts_df
from dbwrapper.crud.read.select_unique_staged_transactions import select_unique_staged_transactions
from dbwrapper.crud.read.select_final_transactions import select_final_transactions
from dbwrapper.crud.create.insert_query_into_table import insert_query_into_table

from dbwrapper.etl.full_load_from_csv import full_load_from_csv

pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
inside dbwrapper.__init__
leaving dbwrapper.__init__


In [2]:
## empty cell
full_load_from_csv(engine)
query = select_final_transactions(engine)
df = query_to_df(query)

In [3]:
summarize('spotify', df)

,sha256,postDate,transactionDate,accountId,amount,description,memo,details,type,category,balance,checkOrSlipNumber
35,4a7c1eca42,2021-05-03,NaT,1,-11.39,Spotify USA 877-7781161 NY 04/30,None,DEBIT,DEBIT_CARD,None,14998.75,None
148,2556790212,2021-06-01,NaT,1,-11.33,Spotify USA 877-7781161 NY 05/31,None,DEBIT,DEBIT_CARD,None,26736.08,None
296,00fabf446c,2021-07-01,NaT,1,-11.33,Spotify USA 877-7781161 NY 06/30,None,DEBIT,DEBIT_CARD,None,24776.85,None
456,f9bcb0b175,2021-08-02,NaT,1,-11.33,Spotify USA 877-7781161 NY 07/31,None,DEBIT,DEBIT_CARD,None,21430.87,None
589,88edccf9a2,2021-09-01,NaT,1,-11.33,Spotify USA 877-7781161 NY 08/31,None,DEBIT,DEBIT_CARD,None,14336.20,None
730,b130442c0e,2021-10-01,NaT,1,-11.33,Spotify USA 877-7781161 NY 09/30,None,DEBIT,DEBIT_CARD,None,13764.99,None
893,409b30e31e,2021-11-01,NaT,1,-11.33,Spotify USA 877-7781161 NY 10/31,None,DEBIT,DEBIT_CARD,None,10224.14,None
1043,2cf0673e26,2021-11-30,NaT,1,-11.33,Spotify USA 877-7781161 NY 11/30,None,DEBIT,DEBIT_CARD,None,3316.27,None
1184,41a9095ce6,2021-12-31,NaT,1,-11.33,Spotify USA 877-7781161 NY 12/31,None,DEBIT,DEBIT_CARD,None,2052.07,None
1292,b6d1e2df24,2022-01-31,NaT,1,-11.33,Spotify USA 877-7781161 NY 01/31,None,DEBIT,DEBIT_CARD,None,4094.57,None


total number of transactions= 25
sum= -281.96999999999997
avg cost per transaction= -11.28
dates= 2023-05-01 00:00:00 to 2021-05-03 00:00:00, total days=-728
avg days between transaction= -29.12
avg cost per month= 11.78890232142857


In [4]:
summarize('publix', df)

,sha256,postDate,transactionDate,accountId,amount,description,memo,details,type,category,balance,checkOrSlipNumber
6,f3688d6af1,2021-04-19,NaT,1,-40.22,PUBLIX SUPER MAR 525 S CLEARWATER FL 120132 04/18,None,DEBIT,DEBIT_CARD,None,954.31,None
11,42b31f8c01,2021-04-21,NaT,1,-77.39,PUBLIX SUPER MAR 3615 TAMPA FL 031951 04/21,None,DEBIT,DEBIT_CARD,None,10589.77,None
21,ed16e05b0b,2021-04-26,NaT,1,-128.53,PUBLIX SUPER MAR 3615 TAMPA FL 543748 04/25,None,DEBIT,DEBIT_CARD,None,12755.34,None
29,577f24db29,2021-04-29,NaT,1,-84.00,PUBLIX SUPER MAR 3615 TAMPA FL 240455 04/29,None,DEBIT,DEBIT_CARD,None,12276.67,None
36,4d91c6a9e0,2021-05-03,NaT,1,-65.88,PUBLIX SUPER MAR 3615 TAMPA FL 751137 05/03,None,DEBIT,DEBIT_CARD,None,10817.96,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3264,f7897c0827,2023-05-08,2023-05-06,2,-16.57,PUBLIX #867,None,None,Sale,Groceries,NaN,None
3277,2310077dc4,2023-05-12,2023-05-11,2,-137.91,PUBLIX #1808,None,None,Sale,Groceries,NaN,None
3295,27efa07df2,2023-05-16,2023-05-15,2,-15.49,PUBLIX #1506,None,None,Sale,Groceries,NaN,None
3296,6e474ca54c,2023-05-16,2023-05-15,2,-60.59,PUBLIX #1808,None,None,Sale,Groceries,NaN,None


total number of transactions= 308
sum= -20178.229999999996
avg cost per transaction= -65.51
dates= 2023-05-16 00:00:00 to 2021-04-19 00:00:00, total days=-757
avg days between transaction= -2.457792207792208
avg cost per month= 811.3141169220606
